# Tabular Record Compilation

### Run this code separately for the training and production datasets

In [5]:
from pathlib import Path
import os
import pandas as pd

# Replace this with the desired output type ('production', 'training')
# output_type = 'training'
output_type = 'production'

base_dir = Path("../../dataset/clean").joinpath(output_type, "pre_production")
years = os.listdir(base_dir)

columns = ['Control Number', 'Cleaned Present Reading']
all_data = []

for year in years:
    files = os.listdir(base_dir / year)
    for file in files:
        month = file.split("_")[0]
        df = pd.read_csv(base_dir / year / file, usecols=columns)
        df['Month_Year'] = f"{year}_{month}"
        all_data.append(df)

# Combine into a single DataFrame
df_all = pd.concat(all_data, ignore_index=True)

# Pivot to wide format
wide_df = df_all.pivot_table(
    index=['Control Number'],
    columns='Month_Year',
    values='Cleaned Present Reading',
).reset_index()

wide_df = wide_df.sort_index(axis=1)

# Optional: Sort columns so months are in order
print(wide_df.head())


Month_Year  2019_DEC  2020_APR  2020_AUG  2020_DEC  2020_FEB  2020_JAN  \
0                NaN       NaN       NaN       NaN       NaN       NaN   
1                NaN       NaN       NaN       NaN       NaN       NaN   
2                NaN       NaN       NaN       NaN       NaN       NaN   
3                NaN    5465.0       NaN       NaN       NaN       NaN   
4                NaN    3280.0       NaN       NaN       NaN       NaN   

Month_Year  2020_JUL  2020_JUN  2020_MAR  2020_MAY  ...  2024_MAY  2024_NOV  \
0                NaN       NaN       NaN       NaN  ...    2206.0       NaN   
1                NaN       NaN       NaN       NaN  ...       NaN       0.0   
2                NaN       NaN       NaN       NaN  ...     111.0       NaN   
3                NaN       NaN       NaN       NaN  ...       NaN       0.0   
4                NaN       NaN       NaN       NaN  ...    2149.0    2629.0   

Month_Year  2024_OCT  2024_SEP  2025_APR  2025_FEB  2025_JAN  2025_MAR  \
0     

In [6]:
%pip install calendar

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement calendar (from versions: none)
ERROR: No matching distribution found for calendar


In [7]:
# ================================================
# Consolidate Wide Data with Connection Status and Last Reading
# ================================================

import calendar
import os
from utilities import read_csv, save_csv

# ================================================
# 1. Read and process all monthly data for status tracking
# ================================================
status_dfs = []
for year in years:
    year_dir = base_dir / str(year)
    if not year_dir.exists():
        continue
    for file in os.listdir(year_dir):
        if not file.endswith('.csv'):
            continue
        month_year = file.split('_')[1].split('.')[0]  # Extract YYYY from filename
        df = pd.read_csv(year_dir / file, usecols=['Control Number', 'Connection Status'])
        df['Status_Date'] = pd.to_datetime(month_year, format='%Y')
        status_dfs.append(df)

# Combine all status history
status_history = pd.concat(status_dfs).drop_duplicates()

# Get the most recent status for each control number
latest_status = status_history.sort_values('Status_Date').drop_duplicates('Control Number', keep='last')
latest_status = latest_status[['Control Number', 'Connection Status']]

# ================================================
# 2. Merge with existing wide_df
# ================================================
# Merge with the latest status
final_df = wide_df.merge(
    latest_status,
    on='Control Number',
    how='left'
)

# Normalize connection status values
final_df['Connection Status'] = (
    final_df['Connection Status']
    .replace({"Invalid Reading": "Active"})
    .fillna("Unknown")
)

# ------------------------------------------------
# 3. Identify month columns (exclude non-monthly fields)
# ------------------------------------------------
fixed_cols_pre_sort = [col for col in final_df.columns if not col[0].isdigit()]  # Columns that are not YYYY_MON format
month_cols = [col for col in final_df.columns if col not in fixed_cols_pre_sort]

# ------------------------------------------------
# 4. Sort month columns chronologically
# ------------------------------------------------
month_map = {m.upper(): i for i, m in enumerate(calendar.month_abbr) if m}  # {"JAN":1, "FEB":2, ...}

month_cols_sorted = sorted(
    month_cols,
    key=lambda x: (int(x.split('_')[0]), month_map[x.split('_')[1].upper()])
)

# ------------------------------------------------
# 5. Compute the "Last Reading" (latest non-null value per row)
# ------------------------------------------------
final_df["Last Reading"] = final_df[month_cols_sorted].apply(
    lambda row: row.dropna().iloc[-1] if row.dropna().size > 0 else None,
    axis=1
)

# ------------------------------------------------
# 6. Merge account names from master list
# ------------------------------------------------
if output_type == 'training':
    name_df = pd.read_csv("../../dataset/clean/production/semi_clean/cleaned_master.csv")
    final_df = final_df.merge(
        name_df[['Control Number', 'Account Name']],
        on='Control Number',
        how='left'
)

# ------------------------------------------------
# 7. Final column ordering: fixed fields first, then months
# ------------------------------------------------
if output_type == 'training':
    fixed_cols_final = ['Control Number', 'Account Name', 'Connection Status', 'Last Reading']
    final_df = final_df[fixed_cols_final + month_cols_sorted]
else:
    fixed_cols_final = ['Control Number', 'Connection Status', 'Last Reading']
    final_df = final_df[fixed_cols_final + month_cols_sorted]

# ------------------------------------------------
# 8. Save the consolidated output
# ------------------------------------------------
if output_type == 'training':
    final_df = final_df.drop('Account Name', axis = 1)
save_path = f"../../dataset/clean/{output_type}/production/tabular/consolidated.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
save_csv(final_df, save_path)

# ------------------------------------------------
# 9. Preview output for sanity check
# ------------------------------------------------
display(final_df.head())
display(final_df.info())


Control Number Connection Status  Last Reading  2019_DEC  2020_JAN  \
0             0.0            Active           0.0       NaN       NaN   
1           121.0      Disconnected           0.0       NaN       NaN   
2         50964.0            Active         129.0       NaN       NaN   
3        500001.0      Disconnected           0.0       NaN       NaN   
4        500002.0            Active        2782.0       NaN       NaN   

   2020_FEB  2020_MAR  2020_APR  2020_MAY  2020_JUN  ...  2024_AUG  2024_SEP  \
0       NaN       NaN       NaN       NaN       NaN  ...       0.0       NaN   
1       NaN       NaN       NaN       NaN       NaN  ...       NaN       NaN   
2       NaN       NaN       NaN       NaN       NaN  ...       NaN       NaN   
3       NaN       NaN    5465.0       NaN       NaN  ...       0.0       0.0   
4       NaN       NaN    3280.0       NaN       NaN  ...       NaN       NaN   

   2024_OCT  2024_NOV  2024_DEC  2025_JAN  2025_FEB  2025_MAR  2025_APR  \
0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1       NaN       0.0       0.0       0.0       0.0       0.0       0.0   
2       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4    2609.0    2629.0    2629.0    2629.0       NaN       NaN       NaN   

   2025_MAY  
0       NaN  
1       0.0  
2       NaN  
3       0.0  
4    2782.0  

[5 rows x 57 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2170 entries, 0 to 2169
Data columns (total 57 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Control Number     2170 non-null   float64
 1   Connection Status  2170 non-null   object 
 2   Last Reading       2170 non-null   float64
 3   2019_DEC           1141 non-null   float64
 4   2020_JAN           1174 non-null   float64
 5   2020_FEB           1206 non-null   float64
 6   2020_MAR           1226 non-null   float64
 7   2020_APR           1616 non-null   float64
 8   2020_MAY           1238 non-null   float64
 9   2020_JUN           1232 non-null   float64
 10  2020_JUL           1260 non-null   float64
 11  2020_AUG           1292 non-null   float64
 12  2020_SEP           1268 non-null   float64
 13  2020_OCT           1271 non-null   float64
 14  2020_NOV           1271 non-null   float64
 15  2020_DEC           1141 non-null   float64
 16  2022_JAN           1371 

None

In [8]:
# Create csv for disconnected accounts
disconnected_df = final_df[(final_df['Connection Status'] == 'Disconnected') | (final_df['Connection Status'] == 'Unknown')]

filepath = Path("../../dataset/clean/production/production/tabular/")
disconnected_df.to_csv(filepath / "disconnected.csv", index=False)

print('Successfully generated the disconnection csv.')

Successfully generated the disconnection csv.
